In [3]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install tensorflow
!pip install keras

In [27]:
from keras.models import Sequential
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('diabetes_final.csv')
features = df[['Insulin dosage', 'Exercise']].values
glucose_levels = df['Blood Sugar Level'].values.reshape(-1, 1)

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
glucose_scaled = scaler.fit_transform(glucose_levels)

# Time-series data preparation
n_steps = 3  # You might need to adjust this based on your data
X, y = [], []
for i in range(len(df) - n_steps):
    X.append(features_scaled[i:i + n_steps, :])
    y.append(glucose_scaled[i + n_steps, 0])

X, y = np.array(X), np.array(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LSTM Model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, features.shape[1])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

model.save("lstm_model.h5")


Epoch 1/50
4/4 [==============================] - 1s 63ms/step - loss: 0.2489 - val_loss: 0.2128
Epoch 2/50
4/4 [==============================] - 0s 10ms/step - loss: 0.2201 - val_loss: 0.1860
Epoch 3/50
4/4 [==============================] - 0s 10ms/step - loss: 0.1947 - val_loss: 0.1619
Epoch 4/50
4/4 [==============================] - 0s 9ms/step - loss: 0.1720 - val_loss: 0.1402
Epoch 5/50
4/4 [==============================] - 0s 8ms/step - loss: 0.1498 - val_loss: 0.1213
Epoch 6/50
4/4 [==============================] - 0s 8ms/step - loss: 0.1330 - val_loss: 0.1042
Epoch 7/50
4/4 [==============================] - 0s 8ms/step - loss: 0.1169 - val_loss: 0.0899
Epoch 8/50
4/4 [==============================] - 0s 15ms/step - loss: 0.1024 - val_loss: 0.0789
Epoch 9/50
4/4 [==============================] - 0s 10ms/step - loss: 0.0944 - val_loss: 0.0713
Epoch 10/50
4/4 [==============================] - 0s 9ms/step - loss: 0.0878 - val_loss: 0.0679
Epoch 11/50
4/4 [=================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:

from tensorflow.keras.models import load_model

loaded_model = load_model("lstm_model.h5")

df['Date and Time'] = pd.to_datetime(df['Date and Time'])
df.sort_values('Date and Time', inplace=True)

# Extract the last n_steps data to use as input for prediction
last_data = df[['Insulin dosage', 'Exercise']].tail(n_steps).values



# Reshape and predict the future values
future_X = last_data.reshape(1, n_steps, -1)
future_prediction_scaled = model.predict(future_X)

# Inverse transform to get the actual predicted glucose values
future_prediction = scaler.inverse_transform(future_prediction_scaled)

print("Predicted Blood Sugar Levels for Tomorrow and Day After:")
a = future_prediction[0, 0]
print("Tomorrow:", a)

flag = 0

if a > 130:
  flag = 1
elif a < 80:
  flag = 1





1/1 [==============================] - 0s 114ms/step
Predicted Blood Sugar Levels for Tomorrow and Day After:
Tomorrow: 104.83006


In [11]:
from keras.layers import Input, Dense
from keras.models import Model
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load your dataset
# Adjust 'your_dataset.csv' with the actual file path or URL of your dataset
df = pd.read_csv('diabetes_final.csv')

# Extract features (excluding 'datetime' column)
features = df[['Insulin dosage', 'Blood Sugar Level', 'Exercise']].values

# Standardize features
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

# Define autoencoder architecture
input_dim = features_standardized.shape[1]
encoding_dim = 2  # Choose a lower-dimensional representation for encoding

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation='relu')(input_layer)
decoder = Dense(input_dim, activation='sigmoid')(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder on normal patterns (no anomalies)
autoencoder.fit(features_standardized, features_standardized, epochs=50, batch_size=32, shuffle=True, validation_split=0.2)

# Reconstruct the features using the trained autoencoder
reconstructed_features = autoencoder.predict(features_standardized)

model.save("autoencoder_model.h5")
# Calculate reconstruction error (mean squared error between original and reconstructed features)
mse = np.mean(np.square(features_standardized - reconstructed_features), axis=1)

# Set a threshold for anomaly detection (adjust as needed)
threshold = 0.1

# Identify anomalies based on the threshold
anomalies = mse > threshold

# Add 'flag' column to the DataFrame
df['flag'] = 0
df.loc[anomalies, 'flag'] = 1

# Display flagged rows
flagged_rows = df[df['flag'] == 1]
print(flagged_rows)



Epoch 1/50
4/4 [==============================] - 1s 62ms/step - loss: 1.3235 - val_loss: 1.2629
Epoch 2/50
4/4 [==============================] - 0s 14ms/step - loss: 1.3206 - val_loss: 1.2595
Epoch 3/50
4/4 [==============================] - 0s 13ms/step - loss: 1.3176 - val_loss: 1.2562
Epoch 4/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3148 - val_loss: 1.2529
Epoch 5/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3119 - val_loss: 1.2496
Epoch 6/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3090 - val_loss: 1.2464
Epoch 7/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3064 - val_loss: 1.2431
Epoch 8/50
4/4 [==============================] - 0s 13ms/step - loss: 1.3035 - val_loss: 1.2400
Epoch 9/50
4/4 [==============================] - 0s 12ms/step - loss: 1.3007 - val_loss: 1.2368
Epoch 10/50
4/4 [==============================] - 0s 12ms/step - loss: 1.2980 - val_loss: 1.2336
Epoch 11/50
4/4 [============